In [1]:
import torch
from mmcv.ops import MultiScaleDeformableAttention

/home/qngo/venvs/orienternet/lib/python3.9/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


In [19]:
# Define the module
embed_dims = 256  # Embedding dimension
num_heads = 8
num_levels = 2  # Two levels for multi-scale attention
num_points = 4

attention = MultiScaleDeformableAttention(
    embed_dims=embed_dims,
    num_heads=num_heads,
    num_levels=num_levels,
    num_points=num_points,
    batch_first=True  # Assuming batch dimension comes first
)

# Input tensors
bs = 3  # Batch size
h, w = 512, 512  # Spatial dimensions of the original feature map

# Query and key-value tensors
query = torch.rand(bs, h * w, embed_dims)  # Query reshaped to (batch, num_query, embed_dims)
key_value_layer1 = torch.rand(bs, (h // 2) * (w // 2), embed_dims)  # First layer: Half resolution
key_value_layer2 = torch.rand(bs, h * w, embed_dims)  # Second layer: Original resolution

# Combine key and value tensors
key_value = torch.cat([key_value_layer1, key_value_layer2], dim=1)  # Concatenate along the query dimension

# Spatial shapes for two layers
spatial_shapes = torch.tensor([
    [h // 2, w // 2],  # First layer (reduced resolution)
    [h, w]             # Second layer (original resolution)
], dtype=torch.int32)  # Shape: (num_levels, 2)

# Reference points (normalized) for two levels
reference_points = torch.rand(bs, query.size(1), num_levels, 2)  # Random points for 2 levels

# Level start indices (start positions for each level in the concatenated key_value tensor)
level_start_index = torch.tensor([
    0,                              # Start of level 0 (first layer)
    (h // 2) * (w // 2)             # Start of level 1 (second layer)
], dtype=torch.int32)

# Forward pass through the attention module
output = attention.forward(
    query=query,
    key=key_value,
    value=key_value,
    reference_points=reference_points,
    spatial_shapes=spatial_shapes,
    level_start_index=level_start_index
)

# Output
print("Output shape:", output.shape)


Output shape: torch.Size([3, 262144, 256])


In [21]:
key_value.size()

torch.Size([3, 327680, 256])

In [22]:
key_value_layer1.size()

torch.Size([3, 65536, 256])